In [1]:
!pip install sweetviz
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.1-py2.py3-none-any.whl size=257484150 sha256=903e890e33b6c698342ec3230818ed87d5329926ff1eea5f7695ec686d8b6089
  Stored in directory: /root/.cache/pip/wheels/d9/9b/ca/7345b72d17e1e17da37239d70631c3214ec9e541b0c9e700e2
Successfully built h2o


In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Data Preprocessing and Cleaning**

In [4]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/CMPE-255/Assignment 5/Data Set/Task-2/AirQualityUCI.csv', delimiter=';', decimal=',')

# Display the first few rows of the dataset
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


In [5]:
# Remove unnecessary columns
data_cleaned = data.drop(columns=['Unnamed: 15', 'Unnamed: 16'])

# Check for missing values
missing_values = data_cleaned.isnull().sum()

missing_values


Date             114
Time             114
CO(GT)           114
PT08.S1(CO)      114
NMHC(GT)         114
C6H6(GT)         114
PT08.S2(NMHC)    114
NOx(GT)          114
PT08.S3(NOx)     114
NO2(GT)          114
PT08.S4(NO2)     114
PT08.S5(O3)      114
T                114
RH               114
AH               114
dtype: int64

In [6]:
# Drop rows with missing Date and Time values
data_cleaned = data_cleaned.dropna(subset=['Date', 'Time'])

# Convert Date and Time columns to a single datetime format
data_cleaned['Datetime'] = pd.to_datetime(data_cleaned['Date'] + ' ' + data_cleaned['Time'], format='%d/%m/%Y %H.%M.%S')

# Drop the original Date and Time columns
data_cleaned = data_cleaned.drop(columns=['Date', 'Time'])

# Set Datetime as the index
data_cleaned.set_index('Datetime', inplace=True)

# Display the first few rows of the cleaned dataset
data_cleaned.head()


,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Datetime,,,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [7]:
# Interpolate missing values using linear interpolation
data_interpolated = data_cleaned.interpolate(method='linear')

# Verify if there are any remaining missing values
remaining_missing = data_interpolated.isnull().sum()

remaining_missing

CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64

**Auto EDA using the sweetviz**

In [8]:
import sweetviz as sv

# Create a report
report = sv.analyze(data_interpolated)

# Show the report
report.show_html("/content/drive/MyDrive/CMPE-255/Assignment 5/Data Set/Task-2/AirQualityReport.html")

                                             |          | [  0%]   00:00 -> (? left)

Report /content/drive/MyDrive/CMPE-255/Assignment 5/Data Set/Task-2/AirQualityReport.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


**Clustering and Anomaly Detection**

In [9]:
from sklearn.ensemble import IsolationForest

# Initialize the model
clf = IsolationForest(contamination=0.05)  # setting contamination to 5% to detect anomalies
anomalies = clf.fit_predict(data_interpolated)

# Convert -1 labels to 0 for anomalies and 1 to 0 for normal data
data_interpolated['Anomaly'] = [1 if i == -1 else 0 for i in anomalies]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


**Feature Processing and Selection**

In [10]:
# Assuming we're predicting 'CO(GT)', generate lag features for it
lags = 3
for i in range(1, lags + 1):
    data_interpolated[f'CO(GT)_lag_{i}'] = data_interpolated['CO(GT)'].shift(i)


In [11]:
window_size = 3
data_interpolated['CO(GT)_rolling_mean'] = data_interpolated['CO(GT)'].rolling(window=window_size).mean()
data_interpolated['CO(GT)_rolling_std'] = data_interpolated['CO(GT)'].rolling(window=window_size).std()


In [12]:
from sklearn.ensemble import RandomForestRegressor

# Drop NA values generated due to lags and rolling window operations
data_for_feature_selection = data_interpolated.dropna()

# Initialize a random forest model
rf = RandomForestRegressor(n_estimators=100)
rf.fit(data_for_feature_selection.drop('CO(GT)', axis=1), data_for_feature_selection['CO(GT)'])

# Get feature importance
feature_importances = pd.Series(rf.feature_importances_, index=data_for_feature_selection.drop('CO(GT)', axis=1).columns)
top_features = feature_importances.nlargest(10).index.tolist()


In [13]:
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11939 sha256=8bec42e1e0fd6d08a0250537e35b6f79f00dad9c2915f2033316539b8e69e251
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [14]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpesxctzau
  JVM stdout: /tmp/tmpesxctzau/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpesxctzau/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_no1qdi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Loading Data into H2O

In [15]:
h2o_df = h2o.H2OFrame(data_interpolated)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Train-Test Split

In [16]:
train, test = h2o_df.split_frame(ratios=[.8], seed=1234)


Run H2O's AutoML

In [17]:
from h2o.automl import H2OAutoML

# Assuming we're predicting 'CO(GT)'
y = "CO(GT)"
x = train.columns
x.remove(y)

# Run AutoML
aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=600)
aml.train(x=x, y=y, training_frame=train)


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20231030_80132_model_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    91                 91                          20821                  4            4            4             7             16            13.5934

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.4074438185137677
RMSE: 1.5515939605817521
MAE: 0.41563891736419767
RMSLE: NaN
Mean Residual Deviance: 2.4074438185137677

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 17.146824021324562
RMSE: 4.140872374430847
MAE: 0.6132899117777529
RMSLE: NaN
Mean Residual Deviance: 17.146824021324562

Cross-Validation Metrics Summary: 
                        mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
mae                     0.609835  0.140936    0.608055      0.575963      0.665755      0.406091      0.79331
mean_residual_deviance  17.1984   13.9434     24.8716       7.88116       14.8545       1.60624       36.7784
mse                     17.1984   13.9434     24.8716       7.88116       14.8545       1.60624       36.7784
r2                      0.997141  0.00229815  0.995732      0.99868       0.997561      0.999734      0.993999
residual_deviance       17.1984   13.9434     24.8716       7.88116       14.8545       1.60624       36.7784
rmse                    3.79611   1.8668      4.98715       2.80734       3.85415       1.26738       6.06452
rmsle                   nan       0           nan           nan           nan           nan           nan

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-10-30 08:08:21  40.751 sec  0                  77.5395          59.4632         6012.37
    2023-10-30 08:08:21  40.812 sec  5                  46.2912          35.4645         2142.87
    2023-10-30 08:08:21  40.871 sec  10                 28.2069          21.4648         795.628
    2023-10-30 08:08:21  40.930 sec  15                 17.1394          12.8981         293.759
    2023-10-30 08:08:21  40.989 sec  20                 10.5201          7.73305         110.672
    2023-10-30 08:08:21  41.053 sec  25                 6.67878          4.66167         44.6061
    2023-10-30 08:08:21  41.114 sec  30                 4.58798          2.86167         21.0496
    2023-10-30 08:08:21  41.179 sec  35                 3.38115          1.77704         11.4322
    2023-10-30 08:08:21  41.248 sec  40                 2.78357          1.16545         7.74824
    2023-10-30 08:08:21  41.315 sec  45                 2.43266          0.824874        5.91785
    2023-10-30 08:08:21  41.378 sec  50                 2.24798          0.652691        5.05341
    2023-10-30 08:08:21  41.445 sec  55                 2.05161          0.559727        4.20911
    2023-10-30 08:08:21  41.509 sec  60                 1.94224          0.513697        3.7723
    2023-10-30 08:08:21  41.576 sec  65                 1.84926          0.47968         3.41975
    2023-10-30 08:08:21  41.658 sec  70                 1.79343          0.45834         3.21638
    2023-10-30 08:08:21  41.723 sec  75                 1.74262          0.446035        3.03674
    2023-10-30 08:08:22  41.788 sec  80                 1.71933          0.437759        2.95609
    2023-10-30 08:08:22  41.853 sec  85                 1.64398          0.430023   

**Leaderboard and Model Selection**

In [18]:
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

model_id                                            rmse       mse       mae    rmsle    mean_residual_deviance
GBM_grid_1_AutoML_1_20231030_80132_model_2       4.14087   17.1468  0.61329       nan                   17.1468
XGBoost_grid_1_AutoML_1_20231030_80132_model_3   4.33813   18.8193  0.439404      nan                   18.8193
GBM_5_AutoML_1_20231030_80132                    4.47084   19.9884  0.785095      nan                   19.9884
GBM_1_AutoML_1_20231030_80132                    4.9023    24.0325  1.46729       nan                   24.0325
GBM_3_AutoML_1_20231030_80132                    5.41115   29.2805  0.974317      nan                   29.2805
GBM_4_AutoML_1_20231030_80132                    5.41723   29.3464  1.02921       nan                   29.3464
XGBoost_3_AutoML_1_20231030_80132                5.4173    29.3471  0.971359      nan                   29.3471
XGBoost_grid_1_AutoML_1_20231030_80132_model_2   5.61604   31.5399  0.909998      nan                   

In [19]:
preds = aml.leader.predict(test)


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
